In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Step 1: Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/br23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[7]

# Step 2: Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Step 3: Write the data to a CSV file
with open('dataBRA23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
    


In [2]:
df = pd.read_csv('dataBRA23raw.csv')
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,15/04/2023,Botafogo,2x1,São Paulo,36.9 %,27.9 %,35.2 %
1,15/04/2023,Palmeiras,2x1,Cuiabá,74.0 %,18.6 %,7.4 %
2,15/04/2023,Red Bull Bragantino,2x1,Bahia,56.7 %,23.7 %,19.6 %
3,15/04/2023,Atlético MG,1x2,Vasco,55.1 %,25.9 %,19.0 %
4,15/04/2023,Athletico PR,2x0,Goiás,61.1 %,22.7 %,16.2 %
...,...,...,...,...,...,...,...
330,12/11/2023,Red Bull Bragantino,2x2,Botafogo,46.3 %,28.6 %,25.2 %
331,12/11/2023,Atlético MG,2x1,Goiás,66.4 %,23.9 %,9.7 %
332,12/11/2023,Grêmio,0x1,Corinthians,51.5 %,23.3 %,25.1 %
333,12/11/2023,Bahia,1x1,Athletico PR,36.5 %,26.8 %,36.8 %


In [3]:
#Step 8: cleaning data frame

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
#changes dtype to datetime in first dataframe
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
#creates a new column with the for the competition name

df = df.assign(Comp='Série A')

#separets the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]
#creates new column with competition name

#comparing scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
# create a new column to store the season information in the second data frame
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df['Temp'] = df['Data'].dt.year
#organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)



df.replace({'América MG': 'América-MG',
            'Atlético MG': 'Atlético-MG',
            'Athletico PR': 'Athletico-PR',                   
            }, inplace=True)

#drops unvalid data
df.dropna(axis=0, how='all', inplace=True)
#sort values by date
df = df.sort_values(by='Data')

#atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

#atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
#corrects any possible issue before saving as new file
df = df.dropna()
df = df.drop(index = 0)
df

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_7484\1842171339.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_7484\1842171339.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_7484\1842171339.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.or

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
2,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
3,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
4,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
5,Série A,2023,2023-04-15,Fortaleza,1,1,Internacional,E,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...
333,Série A,2023,2023-11-12,Bahia,1,1,Athletico-PR,E,1,1,331
329,Série A,2023,2023-11-12,Santos,0,0,São Paulo,E,1,1,332
328,Série A,2023,2023-11-12,Vasco,2,1,América-MG,VC,3,0,333
330,Série A,2023,2023-11-12,Red Bull Bragantino,2,2,Botafogo,E,1,1,334


In [4]:

df.to_csv('data/dataBRA23.csv', index=False)
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
2,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
3,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
4,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
5,Série A,2023,2023-04-15,Fortaleza,1,1,Internacional,E,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...
333,Série A,2023,2023-11-12,Bahia,1,1,Athletico-PR,E,1,1,331
329,Série A,2023,2023-11-12,Santos,0,0,São Paulo,E,1,1,332
328,Série A,2023,2023-11-12,Vasco,2,1,América-MG,VC,3,0,333
330,Série A,2023,2023-11-12,Red Bull Bragantino,2,2,Botafogo,E,1,1,334


In [5]:
df.tail(10)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
325,Série A,2023,2023-11-11,Flamengo,1,1,Fluminense,E,1,1,326
326,Série A,2023,2023-11-11,Palmeiras,3,0,Internacional,VC,3,0,327
327,Série A,2023,2023-11-11,Coritiba,1,0,Cruzeiro,VC,3,0,328
332,Série A,2023,2023-11-12,Grêmio,0,1,Corinthians,VV,0,3,329
331,Série A,2023,2023-11-12,Atlético-MG,2,1,Goiás,VC,3,0,330
333,Série A,2023,2023-11-12,Bahia,1,1,Athletico-PR,E,1,1,331
329,Série A,2023,2023-11-12,Santos,0,0,São Paulo,E,1,1,332
328,Série A,2023,2023-11-12,Vasco,2,1,América-MG,VC,3,0,333
330,Série A,2023,2023-11-12,Red Bull Bragantino,2,2,Botafogo,E,1,1,334
334,Série A,2023,2023-11-12,Cuiabá,2,1,Fortaleza,VC,3,0,335
